In [ ]:
def detectmask_predict(frame, face_net, mask_net):
    (height, weight) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224), (104.0, 177.0, 123.0))
    
    face_net.setInput(blob)
    detections = face_net.forward()
    
    faces = []
    locs = []
    preds = []
    
    for n in range(0, detections.shape[2]):
        confidence = detections[0, 0, n, 2]
        
        if confidence > 0.5:
            box = detections[0, 0, n, 3:7] * np.array([weight, height, weight, height])
            (start_x, start_y, end_x, end_y) = box.astype("int")
            
            (start_x, start_y) = (max(0, start_x), max(0, start_y))
            (end_x, end_y) = (min(weight - 1, end_x), min(height - 1, end_y))
            
            face = frame[start_y: end_y, start_x: end_x]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = preprocess_input(img_to_array(face))
            
            faces.append(face)
            locs.append((start_x, start_y, end_x, end_y))
            
    # only make predictions if at least one face detected
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = mask_net.predict(faces, batch_size=batch_s)
        
    # returns a 2-tuple of the face locations
    return (locs, preds)

In [ ]:
prototxt_path = "face_detector/deploy.prototxt"
weights_path = "face_detector/res10_300x300_ssd_iter_140000.caffemodel"
facenet = cv2.dnn.readNet(prototxt_path, weights_path)

In [ ]:
masknet = load_model("mask_detector.model")
